In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns' , 100)
pd.set_option('display.max_rows' , 100)

## Part I: Deal with Datasets
    - sales dataset
    - buidlings dataset

 - Deal with sales dataset

In [2]:
sales = pd.read_csv('nyc-rolling-sales.csv') # (84548, 22)
# Encode BBL in the sales data
BBL = []
for i in range (len(sales)):
    B1 = sales.BOROUGH[i]
    B2 = sales.BLOCK[i]
    L = sales.LOT[i]
    bbl=str(B1)
    if len(str(B2))==5:
        bbl+=str(B2)
    elif len(str(B2))==4:
        bbl+='0'+str(B2)
    elif len(str(B2))==3:
        bbl+='00'+str(B2)
    elif len(str(B2))==2:
        bbl+='000'+str(B2)
    elif len(str(B2))==1:
        bbl+='0000'+str(B2)
    else:
        bbl+='00000'
    if len(str(L))==4:
        bbl+=str(L)
    elif len(str(L))==3:
        bbl+='0'+str(L)
    elif len(str(L))==2:
        bbl+='00'+str(L)
    elif len(str(L))==1:
        bbl+='000'+str(L)
    else:
        bbl+='0000'
    BBL.append(float(bbl))
    
sales['BBL']=BBL

In [3]:
features = ['BBL', 'SALE PRICE']
sales_price = sales[features] # (84548, 2)

# Clean sales_price
# 1. Replace ' -  ' by NaN
sales_price = sales_price.replace(' -  ', np.nan)

# 2. Drop NaN
sales_price = sales_price.dropna() # (69987, 2)

 - Deal with buildings dataset

In [4]:
build3 = pd.read_csv('BK.csv', low_memory=False) # Brooklyn (3)
build3['Borough Number'] = [1 for i in range (len(build3))] 
build2 = pd.read_csv('BX.csv', low_memory=False) # Bronx (2)
build2['Borough Number'] = [1 for i in range (len(build2))] 
build1 = pd.read_csv('MN.csv', low_memory=False) # Manhattan (1)
build1['Borough Number'] = [1 for i in range (len(build1))] 
build4 = pd.read_csv('QN.csv', low_memory=False) # Queens (4)
build4['Borough Number'] = [1 for i in range (len(build4))] 
build5 = pd.read_csv('SI.csv', low_memory=False) # Staten Island (5)
build5['Borough Number'] = [1 for i in range (len(build5))] 

In [5]:
# Only select these features from buildings
buildings_features = ['BBL', 'SchoolDist' , 'ZipCode', 'FireComp', 'LandUse', 'Easements', 'OwnerName', 'LotArea', 
                      'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea', 'GarageArea', 'StrgeArea', 
                      'FactryArea', 'OtherArea', 'AreaSource', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 
                      'LotFront', 'LotDepth', 'BldgFront', 'BldgDepth', 'ProxCode', 'IrrLotCode', 'LotType', 
                      'BsmtCode', 'AssessLand', 'AssessTot', 'YearBuilt', 'YearAlter1', 'YearAlter2', 'BuiltFAR', 
                      'ResidFAR', 'CommFAR', 'FacilFAR', 'XCoord', 'YCoord', 'ZoneMap', 'Borough Number']
# Since the raw data of buildings have too many features, we only pick the ones from the above list
build1 = build1[buildings_features ] # Manhattan, (42958, 43)
build2 = build2[buildings_features ] # Bronx, (89830, 43)
build3 = build3[buildings_features ] # Brooklyn, (277131, 43)
build4 = build4[buildings_features ] # Queens, (324403, 43)
build5 = build5[buildings_features ] # Staten Island, (124048, 43)

 - Understand the data

In [6]:
# Number of unique BBLs in each dataset
print(len(sales_price.BBL.unique()))
print(len(build1.BBL.unique()))
print(len(build2.BBL.unique()))
print(len(build3.BBL.unique()))
print(len(build4.BBL.unique()))
print(len(build5.BBL.unique()))

55488
42958
89830
277131
324403
124048


In [7]:
# Compute common BBL for sales and buildings
sales_drop = sales_price.drop_duplicates(['BBL']) 
build1_drop = build1.drop_duplicates(['BBL']) 
merge_Build1_sales = pd.merge(sales_drop, build1_drop, on='BBL')
print(merge_Build1_sales.shape)

build2_drop = build2.drop_duplicates(['BBL']) 
merge_Build2_sales = pd.merge(sales_drop, build2_drop, on='BBL')
print(merge_Build2_sales.shape)

build3_drop = build3.drop_duplicates(['BBL']) 
merge_Build3_sales = pd.merge(sales_drop, build3_drop, on='BBL')
print(merge_Build3_sales.shape)

build4_drop = build4.drop_duplicates(['BBL']) 
merge_Build4_sales = pd.merge(sales_drop, build4_drop, on='BBL')
print(merge_Build4_sales.shape)

build5_drop = build5.drop_duplicates(['BBL']) 
merge_Build5_sales = pd.merge(sales_drop, build5_drop, on='BBL')
print(merge_Build5_sales.shape)

(2815, 44)
(5120, 44)
(14858, 44)
(11580, 44)
(5152, 44)


## Part 1.5 Visualization of the sales price on 5 boroughs

In [8]:
buildings = pd.concat([build1, build2, build3, build4, build5], axis=0) # (858370, 43)

In [9]:
merge_df = pd.merge(buildings, sales_price, on='BBL') # (52726, 44)

In [10]:
buildings = buildings # Manhattan

# 0. Drop non-indicative or duplicate-info columns
#drop_columns = ['ZipCode', 'Borough Number']
# Don't drop ZipCode
drop_columns = ['Borough Number']
buildings = buildings.drop(columns=drop_columns)

# # 1. Replace ' -  ' by NaN
# sales = sales.replace(' -  ', np.nan)

# 2. Drop NaN
buildings = buildings.dropna()

# 3. Drop catagorical columns with too many categories
too_many_labels = ['FireComp', 'OwnerName', 'ZoneMap']
buildings = buildings.drop(columns=too_many_labels)
zipCode = buildings[['BBL', 'ZipCode']] # this information is useful for Part III of predictions

# 4. Transform the data type to the desired type
num_features = ['LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea', 'GarageArea', 'StrgeArea',
               'FactryArea', 'OtherArea', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront', 'LotDepth',
               'BldgFront', 'BldgDepth', 'AssessLand', 'AssessTot', 'YearAlter1', 'YearAlter2', 'BuiltFAR']
cat_features = ['SchoolDist', 'LandUse', 'Easements', 'AreaSource', 'ProxCode', 'IrrLotCode',
               'LotType', 'BsmtCode']
id_feature = ['BBL']

for f in num_features:
    if buildings[f].dtype != 'float64' or 'int64':
        buildings[f] = buildings[f].astype('float64') #  convert non-numerical type to numerical

for f in cat_features:
    if buildings[f].dtype != 'O':
        buildings[f] = buildings[f].astype('O')

# 5. One-hot encode the categorical features
# if the model does not like too many categories, we will drop 'BUILDING CLASS CATEGORY'
obj_df = buildings[cat_features]
num_df = buildings[num_features]
BBL_df = buildings[id_feature]
cat_df = pd.get_dummies(obj_df, drop_first=True)

buildings = pd.concat([num_df, cat_df, BBL_df], axis=1) # (40695, 65)

In [11]:
# Create a dataframe that has zipcode, BBL, sales price, sales price/sqrt-foot
BBL = merge_df.BBL
Area = merge_df.BldgArea # the total gross area in square feet
Sales = pd.to_numeric(merge_df['SALE PRICE'])
table1 = pd.concat([BBL, Area, Sales], axis=1)

# merge zipCode and table1 to find the corresponding zip code
table1 = pd.merge(zipCode, table1, on='BBL') # merge_df.shape is (7107, 66), zipCode.shape is (40695, 2)

# Compute Per square-footage
perSquFootPrice = pd.DataFrame({'UnitAreaPrice': table1['SALE PRICE'] / table1.BldgArea}) # (7107, 1)
ZipCode = pd.DataFrame({'ZipCode': table1.ZipCode})

perSquFootPrice = perSquFootPrice.replace(np.inf, np.nan)
perSquFootPrice = perSquFootPrice.dropna() # (7070, 1)

table1 = pd.concat([ZipCode, perSquFootPrice], axis=1)

zipMap = pd.read_csv('zipcodes.csv', sep=';')

In [12]:
zipcode = table1.ZipCode.unique()
Results = list(np.zeros((len(zipcode),3)))

for i in range(len(zipcode)):
    x = zipcode[i]
    zip_df = table1.loc[table1.ZipCode == x] # returns a dataframe that has ZipCode == x
    unitP = zip_df['UnitAreaPrice'].mean()
    num = len(zip_df)
    Results[i] = [x, unitP, num]
    
res1_df = pd.DataFrame(Results, columns = ['ZipCode' , 'UnitAreaPrice' , 'Number'])

In [13]:
table1_map = pd.merge(res1_df, zipMap, on='ZipCode') # (42, 5)
# need to read it to csv and plot it in colab
table1_map.to_csv('Visul_5Boroughs_map.csv')

## Part II: Building the Model

Prepare features, that is features in build1 - 5.
1. Build1, Manhattan
2. Build2, Bronx
3. Build3, Brooklyn
4. Build4, Queens
5. Build5, Staten Island

1. Manhattan.
        - We find the best model is random forest regressor with depth 20, wich a test R^2 = 0.92

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

In [15]:
buildings = build1 # Manhattan

# 0. Drop non-indicative or duplicate-info columns
#drop_columns = ['ZipCode', 'Borough Number']
# Don't drop ZipCode
drop_columns = ['Borough Number']
buildings = buildings.drop(columns=drop_columns)

# # 1. Replace ' -  ' by NaN
# sales = sales.replace(' -  ', np.nan)

# 2. Drop NaN
buildings = buildings.dropna()

# 3. Drop catagorical columns with too many categories
too_many_labels = ['FireComp', 'OwnerName', 'ZoneMap']
buildings = buildings.drop(columns=too_many_labels)
zipCode = buildings[['BBL', 'ZipCode']] # this information is useful for Part III of predictions

# 4. Transform the data type to the desired type
num_features = ['LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea', 'GarageArea', 'StrgeArea',
               'FactryArea', 'OtherArea', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront', 'LotDepth',
               'BldgFront', 'BldgDepth', 'AssessLand', 'AssessTot', 'YearAlter1', 'YearAlter2', 'BuiltFAR']
cat_features = ['SchoolDist', 'LandUse', 'Easements', 'AreaSource', 'ProxCode', 'IrrLotCode',
               'LotType', 'BsmtCode']
id_feature = ['BBL']

for f in num_features:
    if buildings[f].dtype != 'float64' or 'int64':
        buildings[f] = buildings[f].astype('float64') #  convert non-numerical type to numerical

for f in cat_features:
    if buildings[f].dtype != 'O':
        buildings[f] = buildings[f].astype('O')

# 5. One-hot encode the categorical features
# if the model does not like too many categories, we will drop 'BUILDING CLASS CATEGORY'
obj_df = buildings[cat_features]
num_df = buildings[num_features]
BBL_df = buildings[id_feature]
cat_df = pd.get_dummies(obj_df, drop_first=True)

buildings = pd.concat([num_df, cat_df, BBL_df], axis=1) # (40695, 65)

In [16]:
# merge data
merge_df = pd.merge(sales_price, buildings, on='BBL') # (7107, 66)

# train and test
train_df, test_df = train_test_split(merge_df, test_size=0.2, random_state=2)

randomforest_reg = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100)

features = list(set(merge_df.columns)-{'SALE PRICE'})

# Deeper Forest, the best model!
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])
print('The test score is ', forest_model.score(test_df[features], test_df['SALE PRICE']))

print('The score on the entire dataset is ', forest_model.score(merge_df[features], merge_df['SALE PRICE']))

The test score is  0.9232386388842991
The score on the entire dataset is  0.901935474001108


In [17]:
# Entire build dataset
merge_df = buildings # before, merge_df is (7107, 66) when training the model, but now is (40695, 65)

# Predicting
features = list(set(merge_df.columns)-{'SALE PRICE'})

randomforest_reg = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100)
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])

result1 = forest_model.predict(merge_df[features]) # this is the model trained before
result1 = pd.DataFrame({'PredPrice': result1})

# Create a dataframe that has zipcode, BBL, sales price, sales price/sqrt-foot
BBL = merge_df.BBL
Area = merge_df.BldgArea # the total gross area in square feet.
table1 = pd.concat([BBL, Area, result1], axis=1)
# merge zipCode and table1 to find the corresponding zip code
table1 = pd.merge(zipCode, table1, on='BBL') # merge_df.shape is (7107, 66), zipCode.shape is (40695, 2)
table1.head() # (7107, 4)
# Compute Per square-footage
perSquFootPrice = pd.DataFrame({'UnitAreaPrice': table1.PredPrice / table1.BldgArea}) # (7107, 1)
ZipCode = pd.DataFrame({'ZipCode': table1.ZipCode})

perSquFootPrice = perSquFootPrice.replace(np.inf, np.nan)
perSquFootPrice = perSquFootPrice.dropna() # (7070, 1)

table1 = pd.concat([ZipCode, perSquFootPrice], axis=1)

zipMap = pd.read_csv('zipcodes.csv', sep=';')

In [18]:
zipcode = table1.ZipCode.unique()
Results = list(np.zeros((len(zipcode),3)))

for i in range(len(zipcode)):
    x = zipcode[i]
    zip_df = table1.loc[table1.ZipCode == x] # returns a dataframe that has ZipCode == x
    unitP = zip_df['UnitAreaPrice'].mean()
    num = len(zip_df)
    Results[i] = [x, unitP, num]
    
res1_df = pd.DataFrame(Results, columns = ['ZipCode' , 'UnitAreaPrice' , 'Number'])

In [19]:
table1_map = pd.merge(res1_df, zipMap, on='ZipCode') # (42, 5)
# need to read it to csv and plot it in colab
table1_map.to_csv('Manhattan_map.csv')

2. Bronx

In [20]:
buildings = build2 # Bronx

# 0. Drop non-indicative or duplicate-info columns
#drop_columns = ['ZipCode', 'Borough Number']
# Don't drop ZipCode
drop_columns = ['Borough Number']
buildings = buildings.drop(columns=drop_columns)

# # 1. Replace ' -  ' by NaN
# sales = sales.replace(' -  ', np.nan)

# 2. Drop NaN
buildings = buildings.dropna()

# 3. Drop catagorical columns with too many categories
too_many_labels = ['FireComp', 'OwnerName', 'ZoneMap']
buildings = buildings.drop(columns=too_many_labels)
zipCode = buildings[['BBL', 'ZipCode']] # this information is useful for Part III of predictions

# 4. Transform the data type to the desired type
num_features = ['LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea', 'GarageArea', 'StrgeArea',
               'FactryArea', 'OtherArea', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront', 'LotDepth',
               'BldgFront', 'BldgDepth', 'AssessLand', 'AssessTot', 'YearAlter1', 'YearAlter2', 'BuiltFAR']
cat_features = ['SchoolDist', 'LandUse', 'Easements', 'AreaSource', 'ProxCode', 'IrrLotCode',
               'LotType', 'BsmtCode']
id_feature = ['BBL']

for f in num_features:
    if buildings[f].dtype != 'float64' or 'int64':
        buildings[f] = buildings[f].astype('float64') #  convert non-numerical type to numerical

for f in cat_features:
    if buildings[f].dtype != 'O':
        buildings[f] = buildings[f].astype('O')

# 5. One-hot encode the categorical features
# if the model does not like too many categories, we will drop 'BUILDING CLASS CATEGORY'
obj_df = buildings[cat_features]
num_df = buildings[num_features]
BBL_df = buildings[id_feature]
cat_df = pd.get_dummies(obj_df, drop_first=True)

buildings = pd.concat([num_df, cat_df, BBL_df], axis=1) # (40695, 65)

In [21]:
# merge data
merge_df = pd.merge(sales_price, buildings, on='BBL') # (7107, 66)

# train and test
train_df, test_df = train_test_split(merge_df, test_size=0.2, random_state=2)


# Linear
ridge = Ridge(alpha=1.0, fit_intercept=True, normalize=True)
features = list(set(train_df.columns)-{'SALE PRICE'})

# 
ridge_model = ridge.fit(train_df[features], train_df['SALE PRICE'])
print('The test score is ', ridge_model.score(test_df[features], test_df['SALE PRICE']))

print('The score on the entire dataset is ', ridge_model.score(merge_df[features], merge_df['SALE PRICE']))

The test score is  0.06319608897726103
The score on the entire dataset is  0.14628867240642696


In [22]:
# Entire build dataset
merge_df = buildings # before, merge_df is (7107, 66) when training the model, but now is (40695, 65)

# Predicting
features = list(set(merge_df.columns)-{'SALE PRICE'})

randomforest_reg = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100)
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])

result1 = forest_model.predict(merge_df[features]) # this is the model trained before
result1 = pd.DataFrame({'PredPrice': result1})

# Create a dataframe that has zipcode, BBL, sales price, sales price/sqrt-foot
BBL = merge_df.BBL
Area = merge_df.BldgArea # the total gross area in square feet.
table1 = pd.concat([BBL, Area, result1], axis=1)
# merge zipCode and table1 to find the corresponding zip code
table1 = pd.merge(zipCode, table1, on='BBL') # merge_df.shape is (7107, 66), zipCode.shape is (40695, 2)
table1.head() # (7107, 4)
# Compute Per square-footage
perSquFootPrice = pd.DataFrame({'UnitAreaPrice': table1.PredPrice / table1.BldgArea}) # (7107, 1)
ZipCode = pd.DataFrame({'ZipCode': table1.ZipCode})

perSquFootPrice = perSquFootPrice.replace(np.inf, np.nan)
perSquFootPrice = perSquFootPrice.dropna() # (7070, 1)

table1 = pd.concat([ZipCode, perSquFootPrice], axis=1)

zipMap = pd.read_csv('zipcodes.csv', sep=';')

In [23]:
zipcode = table1.ZipCode.unique()
Results = list(np.zeros((len(zipcode),3)))

for i in range(len(zipcode)):
    x = zipcode[i]
    zip_df = table1.loc[table1.ZipCode == x] # returns a dataframe that has ZipCode == x
    unitP = zip_df['UnitAreaPrice'].mean()
    num = len(zip_df)
    Results[i] = [x, unitP, num]
    
res1_df = pd.DataFrame(Results, columns = ['ZipCode' , 'UnitAreaPrice' , 'Number'])

In [24]:
table1_map = pd.merge(res1_df, zipMap, on='ZipCode') # (42, 5)
# need to read it to csv and plot it in colab
table1_map.to_csv('Bronx_map.csv')

3. Brooklyn

In [25]:
buildings = build3 # Brooklyn

# 0. Drop non-indicative or duplicate-info columns
#drop_columns = ['ZipCode', 'Borough Number']
# Don't drop ZipCode
drop_columns = ['Borough Number']
buildings = buildings.drop(columns=drop_columns)

# # 1. Replace ' -  ' by NaN
# sales = sales.replace(' -  ', np.nan)

# 2. Drop NaN
buildings = buildings.dropna()

# 3. Drop catagorical columns with too many categories
too_many_labels = ['FireComp', 'OwnerName', 'ZoneMap']
buildings = buildings.drop(columns=too_many_labels)
zipCode = buildings[['BBL', 'ZipCode']] # this information is useful for Part III of predictions

# 4. Transform the data type to the desired type
num_features = ['LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea', 'GarageArea', 'StrgeArea',
               'FactryArea', 'OtherArea', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront', 'LotDepth',
               'BldgFront', 'BldgDepth', 'AssessLand', 'AssessTot', 'YearAlter1', 'YearAlter2', 'BuiltFAR']
cat_features = ['SchoolDist', 'LandUse', 'Easements', 'AreaSource', 'ProxCode', 'IrrLotCode',
               'LotType', 'BsmtCode']
id_feature = ['BBL']

for f in num_features:
    if buildings[f].dtype != 'float64' or 'int64':
        buildings[f] = buildings[f].astype('float64') #  convert non-numerical type to numerical

for f in cat_features:
    if buildings[f].dtype != 'O':
        buildings[f] = buildings[f].astype('O')

# 5. One-hot encode the categorical features
# if the model does not like too many categories, we will drop 'BUILDING CLASS CATEGORY'
obj_df = buildings[cat_features]
num_df = buildings[num_features]
BBL_df = buildings[id_feature]
cat_df = pd.get_dummies(obj_df, drop_first=True)

buildings = pd.concat([num_df, cat_df, BBL_df], axis=1) # (40695, 65)

In [26]:
# merge data
merge_df = pd.merge(sales_price, buildings, on='BBL') # (7107, 66)

# train and test
train_df, test_df = train_test_split(merge_df, test_size=0.2, random_state=2)

features = list(set(merge_df.columns)-{'SALE PRICE'})

randomforest_reg = RandomForestRegressor(max_depth=65, random_state=0, n_estimators=40)

# Deeper Forest, the best model!
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])
print('The test score is ', forest_model.score(test_df[features], test_df['SALE PRICE']))

print('The score on the entire dataset is ', forest_model.score(merge_df[features], merge_df['SALE PRICE']))

The test score is  -0.01301712733809568
The score on the entire dataset is  0.6883358035144499


In [27]:
# Entire build dataset
merge_df = buildings # before, merge_df is (7107, 66) when training the model, but now is (40695, 65)

# Predicting
features = list(set(merge_df.columns)-{'SALE PRICE'})

randomforest_reg = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100)
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])

result1 = forest_model.predict(merge_df[features]) # this is the model trained before
result1 = pd.DataFrame({'PredPrice': result1})

# Create a dataframe that has zipcode, BBL, sales price, sales price/sqrt-foot
BBL = merge_df.BBL
Area = merge_df.BldgArea # the total gross area in square feet.
table1 = pd.concat([BBL, Area, result1], axis=1)
# merge zipCode and table1 to find the corresponding zip code
table1 = pd.merge(zipCode, table1, on='BBL') # merge_df.shape is (7107, 66), zipCode.shape is (40695, 2)
table1.head() # (7107, 4)
# Compute Per square-footage
perSquFootPrice = pd.DataFrame({'UnitAreaPrice': table1.PredPrice / table1.BldgArea}) # (7107, 1)
ZipCode = pd.DataFrame({'ZipCode': table1.ZipCode})

perSquFootPrice = perSquFootPrice.replace(np.inf, np.nan)
perSquFootPrice = perSquFootPrice.dropna() # (7070, 1)

table1 = pd.concat([ZipCode, perSquFootPrice], axis=1)

zipMap = pd.read_csv('zipcodes.csv', sep=';')

In [28]:
zipcode = table1.ZipCode.unique()
Results = list(np.zeros((len(zipcode),3)))

for i in range(len(zipcode)):
    x = zipcode[i]
    zip_df = table1.loc[table1.ZipCode == x] # returns a dataframe that has ZipCode == x
    unitP = zip_df['UnitAreaPrice'].mean()
    num = len(zip_df)
    Results[i] = [x, unitP, num]
    
res1_df = pd.DataFrame(Results, columns = ['ZipCode' , 'UnitAreaPrice' , 'Number'])

In [29]:
table1_map = pd.merge(res1_df, zipMap, on='ZipCode') # (42, 5)
# need to read it to csv and plot it in colab
table1_map.to_csv('Brooklyn_map.csv')

4. Queens

In [30]:
buildings = build4 # Queens

# 0. Drop non-indicative or duplicate-info columns
#drop_columns = ['ZipCode', 'Borough Number']
# Don't drop ZipCode
drop_columns = ['Borough Number']
buildings = buildings.drop(columns=drop_columns)

# # 1. Replace ' -  ' by NaN
# sales = sales.replace(' -  ', np.nan)

# 2. Drop NaN
buildings = buildings.dropna()

# 3. Drop catagorical columns with too many categories
too_many_labels = ['FireComp', 'OwnerName', 'ZoneMap']
buildings = buildings.drop(columns=too_many_labels)
zipCode = buildings[['BBL', 'ZipCode']] # this information is useful for Part III of predictions

# 4. Transform the data type to the desired type
num_features = ['LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea', 'GarageArea', 'StrgeArea',
               'FactryArea', 'OtherArea', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront', 'LotDepth',
               'BldgFront', 'BldgDepth', 'AssessLand', 'AssessTot', 'YearAlter1', 'YearAlter2', 'BuiltFAR']
cat_features = ['SchoolDist', 'LandUse', 'Easements', 'AreaSource', 'ProxCode', 'IrrLotCode',
               'LotType', 'BsmtCode']
id_feature = ['BBL']

for f in num_features:
    if buildings[f].dtype != 'float64' or 'int64':
        buildings[f] = buildings[f].astype('float64') #  convert non-numerical type to numerical

for f in cat_features:
    if buildings[f].dtype != 'O':
        buildings[f] = buildings[f].astype('O')

# 5. One-hot encode the categorical features
# if the model does not like too many categories, we will drop 'BUILDING CLASS CATEGORY'
obj_df = buildings[cat_features]
num_df = buildings[num_features]
BBL_df = buildings[id_feature]
cat_df = pd.get_dummies(obj_df, drop_first=True)

buildings = pd.concat([num_df, cat_df, BBL_df], axis=1) # (40695, 65)

In [31]:
# merge data
merge_df = pd.merge(sales_price, buildings, on='BBL') # (7107, 66)

# train and test
train_df, test_df = train_test_split(merge_df, test_size=0.2, random_state=2)

features = list(set(merge_df.columns)-{'SALE PRICE'})

randomforest_reg = RandomForestRegressor(max_depth=100, random_state=0, n_estimators=60)

# Deeper Forest, the best model!
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])
print('The test score is ', forest_model.score(test_df[features], test_df['SALE PRICE']))

print('The score on the entire dataset is ', forest_model.score(merge_df[features], merge_df['SALE PRICE']))

The test score is  0.2608671902515134
The score on the entire dataset is  0.837152182420502


In [32]:
# Entire build dataset
merge_df = buildings # before, merge_df is (7107, 66) when training the model, but now is (40695, 65)

# Predicting
features = list(set(merge_df.columns)-{'SALE PRICE'})

randomforest_reg = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100)
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])

result1 = forest_model.predict(merge_df[features]) # this is the model trained before
result1 = pd.DataFrame({'PredPrice': result1})

# Create a dataframe that has zipcode, BBL, sales price, sales price/sqrt-foot
BBL = merge_df.BBL
Area = merge_df.BldgArea # the total gross area in square feet.
table1 = pd.concat([BBL, Area, result1], axis=1)
# merge zipCode and table1 to find the corresponding zip code
table1 = pd.merge(zipCode, table1, on='BBL') # merge_df.shape is (7107, 66), zipCode.shape is (40695, 2)
table1.head() # (7107, 4)
# Compute Per square-footage
perSquFootPrice = pd.DataFrame({'UnitAreaPrice': table1.PredPrice / table1.BldgArea}) # (7107, 1)
ZipCode = pd.DataFrame({'ZipCode': table1.ZipCode})

perSquFootPrice = perSquFootPrice.replace(np.inf, np.nan)
perSquFootPrice = perSquFootPrice.dropna() # (7070, 1)

table1 = pd.concat([ZipCode, perSquFootPrice], axis=1)

zipMap = pd.read_csv('zipcodes.csv', sep=';')

In [33]:
zipcode = table1.ZipCode.unique()
Results = list(np.zeros((len(zipcode),3)))

for i in range(len(zipcode)):
    x = zipcode[i]
    zip_df = table1.loc[table1.ZipCode == x] # returns a dataframe that has ZipCode == x
    unitP = zip_df['UnitAreaPrice'].mean()
    num = len(zip_df)
    Results[i] = [x, unitP, num]
    
res1_df = pd.DataFrame(Results, columns = ['ZipCode' , 'UnitAreaPrice' , 'Number'])

In [34]:
table1_map = pd.merge(res1_df, zipMap, on='ZipCode') # (42, 5)
# need to read it to csv and plot it in colab
table1_map.to_csv('Queens_map.csv')

5. Staten Island

In [35]:
buildings = build5 # Staten Island

# 0. Drop non-indicative or duplicate-info columns
#drop_columns = ['ZipCode', 'Borough Number']
# Don't drop ZipCode
drop_columns = ['Borough Number']
buildings = buildings.drop(columns=drop_columns)

# # 1. Replace ' -  ' by NaN
# sales = sales.replace(' -  ', np.nan)

# 2. Drop NaN
buildings = buildings.dropna()

# 3. Drop catagorical columns with too many categories
too_many_labels = ['FireComp', 'OwnerName', 'ZoneMap']
buildings = buildings.drop(columns=too_many_labels)
zipCode = buildings[['BBL', 'ZipCode']] # this information is useful for Part III of predictions

# 4. Transform the data type to the desired type
num_features = ['LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea', 'GarageArea', 'StrgeArea',
               'FactryArea', 'OtherArea', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront', 'LotDepth',
               'BldgFront', 'BldgDepth', 'AssessLand', 'AssessTot', 'YearAlter1', 'YearAlter2', 'BuiltFAR']
cat_features = ['SchoolDist', 'LandUse', 'Easements', 'AreaSource', 'ProxCode', 'IrrLotCode',
               'LotType', 'BsmtCode']
id_feature = ['BBL']

for f in num_features:
    if buildings[f].dtype != 'float64' or 'int64':
        buildings[f] = buildings[f].astype('float64') #  convert non-numerical type to numerical

for f in cat_features:
    if buildings[f].dtype != 'O':
        buildings[f] = buildings[f].astype('O')

# 5. One-hot encode the categorical features
# if the model does not like too many categories, we will drop 'BUILDING CLASS CATEGORY'
obj_df = buildings[cat_features]
num_df = buildings[num_features]
BBL_df = buildings[id_feature]
cat_df = pd.get_dummies(obj_df, drop_first=True)

buildings = pd.concat([num_df, cat_df, BBL_df], axis=1) # (40695, 65)

In [36]:
# merge data
merge_df = pd.merge(sales_price, buildings, on='BBL') # (7107, 66)

# train and test
train_df, test_df = train_test_split(merge_df, test_size=0.2, random_state=2)

features = list(set(merge_df.columns)-{'SALE PRICE'})

randomforest_reg = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100)

# Deeper Forest, the best model!
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])
print('The test score is ', forest_model.score(test_df[features], test_df['SALE PRICE']))

print('The score on the entire dataset is ', forest_model.score(merge_df[features], merge_df['SALE PRICE']))

The test score is  0.674526769360178
The score on the entire dataset is  0.8068279832122394


In [37]:
# Entire build dataset
merge_df = buildings # before, merge_df is (7107, 66) when training the model, but now is (40695, 65)

# Predicting
features = list(set(merge_df.columns)-{'SALE PRICE'})

randomforest_reg = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=100)
forest_model = randomforest_reg.fit(train_df[features], train_df['SALE PRICE'])

result1 = forest_model.predict(merge_df[features]) # this is the model trained before
result1 = pd.DataFrame({'PredPrice': result1})

# Create a dataframe that has zipcode, BBL, sales price, sales price/sqrt-foot
BBL = merge_df.BBL
Area = merge_df.BldgArea # the total gross area in square feet.
table1 = pd.concat([BBL, Area, result1], axis=1)
# merge zipCode and table1 to find the corresponding zip code
table1 = pd.merge(zipCode, table1, on='BBL') # merge_df.shape is (7107, 66), zipCode.shape is (40695, 2)
table1.head() # (7107, 4)
# Compute Per square-footage
perSquFootPrice = pd.DataFrame({'UnitAreaPrice': table1.PredPrice / table1.BldgArea}) # (7107, 1)
ZipCode = pd.DataFrame({'ZipCode': table1.ZipCode})

perSquFootPrice = perSquFootPrice.replace(np.inf, np.nan)
perSquFootPrice = perSquFootPrice.dropna() # (7070, 1)

table1 = pd.concat([ZipCode, perSquFootPrice], axis=1)

zipMap = pd.read_csv('zipcodes.csv', sep=';')

In [38]:
zipcode = table1.ZipCode.unique()
Results = list(np.zeros((len(zipcode),3)))

for i in range(len(zipcode)):
    x = zipcode[i]
    zip_df = table1.loc[table1.ZipCode == x] # returns a dataframe that has ZipCode == x
    unitP = zip_df['UnitAreaPrice'].mean()
    num = len(zip_df)
    Results[i] = [x, unitP, num]
    
res1_df = pd.DataFrame(Results, columns = ['ZipCode' , 'UnitAreaPrice' , 'Number'])

In [39]:
table1_map = pd.merge(res1_df, zipMap, on='ZipCode') # (42, 5)
# need to read it to csv and plot it in colab
table1_map.to_csv('StatenIsland_map.csv')